### Runpod 설정

In [1]:
import os

# 모델 저장 경로를 네트워크 볼륨(/workspace) 내부로 지정
os.environ["HF_HOME"] = "/workspace/hf_cache"

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install transformers accelerate bitsandbytes openai torch nltk

In [ ]:
# 필수 라이브러리 설치/업데이트
!pip install huggingface_hub

In [ ]:
!pip install bert-score

In [4]:
from huggingface_hub import notebook_login
notebook_login()

---

In [7]:
# BERT Score 계산
from bert_score import score

def calculate_bertscore(reference, candidate):
    refs = [reference]
    cands = [candidate]

    # score 함수 호출
    # lang="ko": 한국어 처리에 적합한 다국어 모델(mBERT 등)을 자동으로 선택
    # verbose=False: 진행 과정 출력 생략
    P, R, F1 = score(cands, refs, lang="ko", verbose=False)

    # 결과값은 Tensor 형태이므로 .item()으로 숫자만 추출
    return {
        "Precision": round(P.item(), 4),
        "Recall": round(R.item(), 4),
        "F1": round(F1.item(), 4)
    }

### GPT 모델

In [ ]:
import time
import json
import openai

In [5]:
def generate_summary_openai(model_name, prompt):
    start_time = time.perf_counter()
    ttft = 0
    generated_tokens = 0
    content = ""
    first_token_received = False
    
    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[{"role": "system", "content": prompt}],
            temperature=0.1,
            stream=True,                              # 토큰 단위로 수신
            stream_options={"include_usage": True}    # 마지막 chunk에 토큰 정보 포함
        )
        
        for chunk in response:
            # 첫번째 토큰이 들어오는 시점 확인 (ttft)
            if not first_token_received and chunk.choices and chunk.choices[0].delta.content:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            # 내용 누적
            if chunk.choices and chunk.choices[0].delta.content:
                content += chunk.choices[0].delta.content
            
            # 토큰 사용량 확인
            if chunk.usage is not None:
                generated_tokens = chunk.usage.completion_tokens
        
        end_time = time.perf_counter()
        total_duration = end_time - start_time
        
        # tps 계산 (생성된 토큰 수 / 전체 소요 시간)
        tps = generated_tokens / total_duration if total_duration > 0 else 0

        result_json = json.loads(content)
        
        metrics = {
                "ttft": round(ttft, 3),
                "tps": round(tps, 2),
                "total_tokens": generated_tokens
            }

        return result_json, metrics
    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}"

### sllm

In [9]:
import re
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

In [10]:
def generate_summary_sllm(model_name, prompt):
    try:
        # 모델 및 토크나이저 로드
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        messages = [{"role": "system", "content": prompt}]
        inputs = tokenizer.apply_chat_template(
            messages, 
            tokenize=True, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)

        # 스트리머 설정
        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
        
        # 측정 변수 초기화
        start_time = time.perf_counter()
        ttft = 0
        first_token_received = False
        full_content = ""
        generated_token_count = 0

        # 생성 프로세스를 별도 스레드에서 실행
        generation_kwargs = dict(
            input_ids=inputs,
            streamer=streamer,
            max_new_tokens=512,
            repetition_penalty=1.2,
            do_sample=False,
            temperature=0.1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )
        
        thread = Thread(target=model.generate, kwargs=generation_kwargs)
        thread.start()

        # 토큰을 하나씩 읽으며 TTFT 측정
        for new_text in streamer:
            if not first_token_received and len(new_text) > 0:
                ttft = time.perf_counter() - start_time
                first_token_received = True
            
            full_content += new_text

        thread.join()
        end_time = time.perf_counter()
        
        total_duration = end_time - start_time
        generated_token_count = len(tokenizer.encode(full_content))
        tps = generated_token_count / total_duration if total_duration > 0 else 0

        # JSON 추출 및 파싱
        try:
            json_match = re.search(r'(\{.*\})', full_content, re.DOTALL)
            if json_match:
                content = json.loads(json_match.group(1))
            else:
                content = {"error": "JSON 형식을 찾을 수 없음", "raw": full_content}
        except json.JSONDecodeError:
            content = {"error": "JSON 파싱 실패", "raw": full_content}

        # 리소스 정리
        # del model, tokenizer, inputs
        # torch.cuda.empty_cache()

        metrics = {
            "ttft": round(ttft, 3),
            "tps": round(tps, 2),
            "total_tokens": generated_token_count
        }

        return content, metrics

    except Exception as e:
        return f"{model_name} 호출 중 오류 발생 : {e}", None

In [25]:
import os
import json
import glob
import pandas as pd

# 파일 경로 설정
testset_dir = "./testsets"
json_files = glob.glob(os.path.join(testset_dir, "*.json"))

results_list = []

# 파일 루프 시작
for file_path in json_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)
        
    for item in data_list:
        # 데이터 추출
        script = item.get('consulting_content', "")
        summary_ref = item['instructions'][0]['data'][0]['output']

        print(f"테스트 실행 중: {item.get('source_id', 'Unknown ID')}")

        # 모델 테스트
        # prompt 생성
        system_prompt = f"""
            상담 스크립트를 바탕으로 아래 JSON 형식에 맞춰 응답하세요

            ### 제약 사항
            1. 출력은 JSON 데이터만 허용합니다
            2. JSON 외의 서론, 결론, 마크다운 기호(```), 설명은 절대 포함하지 마세요

            ### 상담 스크립트
            {script}

            ### 출력 형식 (JSON)
            {{
                "title": "상담의 핵심 주제를 나타내는 간결한 제목 (예: 결제 오류 문의 및 해결)",
                "status": "'진행중', '완료' 중 택일",
                "inquiry": "고객이 문의한 핵심 내용을 1줄로 요약",
                "process": ["상담 과정을 순서대로 요약한 문자열 리스트"],,
                "result": "상담 결과 요약",
                "next_step": "상담 종료 후 상담원이 추가로 할 일 (없으면 '')",
                "transfer_dep": "이관이 필요한 경우 부서명 (없으면 '')",
                "transfer_note": "이관 부서에 전달할 내용 (없으면 '')"
            }}
            """
        
            
        res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct", system_prompt)
        # res, metrics = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", system_prompt)
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)
        # res, metrics = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)
        # res, metrics = generate_summary_openai("gpt-4.1-mini", system_prompt)
        
        if isinstance(res, dict) and 'process' in res:
            process_list = res['process']
            # 요소가 문자열이 아니면 문자열로 변환
            process_txt = " ".join([str(p) for p in process_list])
        else:
            process_txt = f"ID {item.get('id')}에서 유효한 리스트를 찾을 수 없습니다."
    
        bert = calculate_bertscore(summary_ref, process_txt)
        

        # 결과 저장
        results_list.append({
            "id": item.get('source_id'),
            "script": script,
            "res": res,
            "metrics": metrics,
            "bert": bert
        })

테스트 실행 중: 200577


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

테스트 실행 중: 200587


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

테스트 실행 중: 200583


...

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

테스트 실행 중: 200581


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

테스트 실행 중: 200589


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [27]:
import os
import pandas as pd

df_new = pd.json_normalize(results_list)

if 'res.process' in df_new.columns:
    df_new['res.process'] = df_new['res.process'].apply(lambda x: "\n".join(x) if isinstance(x, list) else x)

df_new['model'] = "EXAONE-3.5-7.8B" 

output_file = "evaluation_results.csv"

if not os.path.exists(output_file):
    df_new.to_csv(output_file, index=False, mode='w', encoding='utf-8-sig')
else:
    df_new.to_csv(output_file, index=False, mode='a', encoding='utf-8-sig', header=False)

print(f"데이터가 추가되었습니다: {output_file}")

데이터가 추가되었습니다: evaluation_results.csv


---

In [17]:
script = "상담사: ▲▲카드 ▲▲▲입니다.\n손님: 예 안녕하세요? 어 저기\n상담사: 네 안녕하십니까?\n손님: ▲▲카드 결 결제하는 그\n손님: 카 저기 결 계좌 이체는 월 변경할려 그러거든요.\n상담사: 결제 계좌를 변경하시겠다 그 말씀이신 거예요?\n손님: 예 예.\n상담사: 알겠습니다. 네 ▲▲▲ 님 본인 맞으십니까?\n손님: 네.\n상담사: 결제 계좌가 저희 쪽에 ▲▲은행 계좌로 등록이 되어 있으신데요. 변경하시고자 하시는 은행명과 계좌 번호 말씀해 주시겠어요, 손님?\n손님: 예.\n손님: ▲▲은행이고요.\n상담사: 네 네.\n손님: 예. ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲\n상담사: ▲▲▲\n손님: ▲▲▲▲\n상담사: ▲▲▲번이요? 말씀해주신 계좌 지금 저희 쪽에서 확인 중에 있습니다. 잠시만 기다려주시겠습니까?\n손님: 예.\n상담사: 네, 손님. 기다려 주셔서 감사합니다. ▲▲카드 이용 대금 출금 이체 계좌 등록에 관한 접수 내용 확인하겠습니다. 생년월일 ▲▲년 ▲▲월 ▲▲일, ▲▲▲ 님 명의로 된 ▲▲은행 ▲▲▲▲▲▲▲▲▲▲▲▲▲ 계좌로\n상담사: 하는 내용이 맞으십니까?\n손님: 네.\n상담사: 확인해주셔서 감사합니다. 요청하신 대로 결제 계좌는 말씀해 주신 ▲▲은행 계좌로 변경 등록 처리해드렸습니다. 그렇기때문에 이 번 ▲▲월 ▲▲일 결제 대금부터는 변경해드린 ▲▲은행 계좌에서 출금 처리가 되는 거고요. 타은행으로 결제\n상담사: 이기 때문에 한도는 결제일 다음 날 원복이 됩니다. 요청하신 대로 결제 계좌는 ▲▲은행 계좌에서 ▲▲은행 계좌로 변경 등록 처리 완료해드렸습니다.\n손님: 네.\n상담사: 감사합니다. 그 외에도 저희 쪽에 장기 카드 대출 등을 이용할 수가 있는데 혹시 카드론 사용 계획은 없으신가요? 유동적이라 변동될 수 있어서요.\n손님: 괜찮습니다.\n상담사: 예 감사합니다. 저는 상담사 ▲▲▲이었습니다."
summary_ref = "손님은 카드 대금 결제 계좌 변경을 문의했다. 본인 확인 이후 상담사는 변경을 원하는 은행명과 계좌 번호를 손님에게 물었다. 해당 계좌를 확인한 다음 상담사는 변경 내용을 손님과 재확인하였고, 고객이 동의하여 변경 등록을 정상적으로 완료하였다. 상담사는 추가로 카드론 이용이 가능함을 안내하였으나 손님은 사양하였고 상담은 종료되었다."

In [18]:
system_prompt = f"""
    상담 스크립트를 바탕으로 아래 JSON 형식에 맞춰 응답하세요

    ### 제약 사항
    1. 출력은 JSON 데이터만 허용합니다
    2. JSON 외의 서론, 결론, 마크다운 기호(```), 설명은 절대 포함하지 마세요

    ### 상담 스크립트
    {script}

    ### 출력 형식 (JSON)
    {{
        "title": "상담의 핵심 주제를 나타내는 간결한 제목 (예: 결제 오류 문의 및 해결)",
        "status": "'진행중', '완료' 중 택일",
        "inquiry": "고객이 문의한 핵심 내용을 1줄로 요약",
        "process": ["단계별 상담 과정을 시간 순서대로 나열", "1단계", "2단계", ..],
        "result": "상담 결과 요약",
        "next_step": "상담 종료 후 상담원이 추가로 할 일 (없으면 '')",
        "transfer_dep": "이관이 필요한 경우 부서명 (없으면 '')",
        "transfer_note": "이관 부서에 전달할 내용 (없으면 '')"
    }}
    """

gpt-4.1-mini

In [ ]:
res_gpt_sm, metrics_gpt_sm = generate_summary_openai("gpt-4.1-mini", system_prompt)

In [ ]:
res_gpt_sm

{'title': '결제 계좌 변경 요청 및 처리',
 'status': '완료',
 'inquiry': '결제 계좌를 다른 은행 계좌로 변경하고 싶다는 요청',
 'process': ['고객이 결제 계좌 변경 의사를 밝힘',
  '상담사가 본인 확인 후 현재 등록된 계좌 정보 안내',
  '고객이 변경 희망 은행명과 계좌 번호 제공',
  '상담사가 계좌 정보 확인 및 등록 진행',
  '변경된 계좌 정보와 처리 완료 안내',
  '추가 카드론 이용 의사 확인 및 고객이 거절'],
 'result': '고객의 결제 계좌가 요청한 ▲▲은행 계좌로 변경 등록 완료됨',
 'next_step': '',
 'transfer_dep': '',
 'transfer_note': ''}

In [ ]:
metrics_gpt_sm

{'ttft': 1.196, 'tps': 54.88, 'total_tokens': 189}

In [ ]:
process_txt = " ".join(res_gpt_sm['process'])
bert_gpt_sm = calculate_bertscore(summary_ref, process_txt)
bert_gpt_sm

'고객이 결제 계좌 변경 의사를 밝힘 상담사가 본인 확인 후 현재 등록된 계좌 정보 안내 고객이 변경 희망 은행명과 계좌 번호 제공 상담사가 계좌 정보 확인 및 등록 진행 변경된 계좌 정보와 처리 완료 안내 추가 카드론 이용 의사 확인 및 고객이 거절'

gpt-4o

In [ ]:
res_4o, metrics_4o = generate_summary_openai("gpt-4o", system_prompt)

In [ ]:
res_4o

{'title': '결제 계좌 변경 요청',
 'status': '완료',
 'inquiry': '결제 계좌를 다른 은행 계좌로 변경 요청',
 'process': ['고객이 결제 계좌 변경 요청',
  '상담사가 고객 본인 확인',
  '고객이 새로운 은행명과 계좌번호 제공',
  '상담사가 새로운 계좌 정보 확인',
  '상담사가 계좌 변경 처리 완료 안내'],
 'result': '결제 계좌가 고객이 요청한 새로운 은행 계좌로 변경 처리됨',
 'next_step': '',
 'transfer_dep': '',
 'transfer_note': ''}

In [ ]:
metrics_4o

{'ttft': 2.04, 'tps': 47.02, 'total_tokens': 151}

In [ ]:
process_txt = " ".join(res_4o['process'])
bert_4o = calculate_bertscore(summary_ref, process_txt)
bert_4o

'고객이 결제 계좌 변경 요청 상담사가 고객 본인 확인 고객이 새로운 은행명과 계좌번호 제공 상담사가 새로운 계좌 정보 확인 상담사가 계좌 변경 처리 완료 안내'

kanana-nano-2.1b

In [16]:
res_kanana_sm, metrics_kanana_sm = generate_summary_sllm("kakaocorp/kanana-nano-2.1b-instruct", system_prompt)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [17]:
res_kanana_sm

{'title': '결제 계좌 변경 문의',
 'status': '완료',
 'inquiry': '결제 계좌를 다른 은행으로 변경 요청',
 'process': ['1. 고객 인사와 상태 확인',
  '2. 결제 계좌 변경 여부 확인',
  '3. 새로운 은행명과 계좌번호 입력 받음',
  '4. 계좌 정보 확인 및 등록 진행',
  '5. 변경된 계좌로 결제 대금 이체 처리 완료',
  '6. 카드론 사용 계획 질문'],
 'result': '고객의 결제 계좌를 성공적으로 다른 은행으로 변경하고, 관련 대금을 새 계좌로 이체함.',
 'next_step': '',
 'transfer.dep': '',
 'transfer_note': ''}

In [ ]:
metrics_kanana_sm

In [ ]:
process_txt = " ".join(res_kanana_sm['process'])
bert_kanana_sm = calculate_bertscore(summary_ref, process_txt)
bert_kanana_sm

kanana-1.5-8b

In [13]:
res_kanana_lg, metrics_kanana_lg = generate_summary_sllm("kakaocorp/kanana-1.5-8b-instruct-2505", system_prompt)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [14]:
res_kanana_lg

{'title': '결제 계좌 변경 신청 및 승인',
 'status': '완료',
 'inquiry': '고객께서 결제 계좌를 다른 ▲▲은행 계좌로 변경하고 싶다고 문의함.',
 'process': ['상담사가 고객에게 인사를 하고 결제 계좌 변경 의도를 확인함.',
  '고객 정보와 변경 희망 계좌 정보를 재확인함.',
  '입력된 계좌 정보를 토대로 실제 적용 가능 여부를 확인하며 안내함.',
  '변경 내역을 최종적으로 확정하여 결제 계좌를 성공적으로 변경 완료함.'],
 'result': '고객님의 결제 계좌를 ▲▲은행 계좌로 정상적으로 변경 등록하였으며, 향후 해당 계좌에서 결제 금액이 출금될 예정임.',
 'next_step': '',
 'transfer_dep': '',
 'transfer_note': ''}

In [ ]:
metrics_kanana_lg

In [ ]:
process_txt = " ".join(res_kanana_lg['process'])
bert_kanana_lg = calculate_bertscore(summary_ref, process_txt)
bert_kanana_lg

EXAONE-3.5-2.4B

In [19]:
res_exaone_sm, metrics_exaone_sm = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct", system_prompt)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [20]:
res_exaone_sm

{'title': '결제 계좌 변경 상담',
 'status': '완료',
 'inquiry': '고객이 기존 ▲▲은행 계좌 대신 다른 은행 계좌로 결제계좌 변경 신청',
 'process': ['상담사와 고객 간 인사 및 환영 인사',
  '고객이 결제 계좌 변경 의사 표현',
  "상담사가 개인 인증 확인 ('본인 확인')",
  "변경하고자 하는 은행 이름('▲▲은행') 입력 요구",
  "은행 계좌 정보 제공 받음 ('▲▲**번호**')",
  "확인 절차 진행 ('출금 이체 계좌 등록 확인')",
  "변경된 계좌 정보 반영하여 안내 ('▲▲은행 계좌로 변경됨')",
  "한도 재설정 안내 ('다음날 원복 예정')",
  "추가 금융 상품 질문 ('장기 카드 대출 관련')"],
 'result': '결제 계좌 성공적으로 ▲▲은행 계좌로 변경되었으며, 해당 달 이후 모든 결제 금액은 새로운 계좌에서 출금 처리됩니다. 장기 카드 대출 관련 질문에는 추후 고려하기로 함.',
 'next_step': '',
 'transfer_dep': '',
 'transfer_note': ''}

In [ ]:
metrics_exaone_sm

In [ ]:
process_txt = " ".join(res_exaone_sm['process'])
bert_exaone_sm = calculate_bertscore(summary_ref, process_txt)
bert_exaone_sm

EXAONE-3.5-7.8B

In [23]:
res_exaone_lg, metrics_exaone_lg = generate_summary_sllm("LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct", system_prompt)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

...

model-00001-of-00007.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [24]:
res_exaone_lg

{'error': 'JSON 파싱 실패',
 'raw': '```json\n{\n    "title": "장기 카드론 금액 및 이자 상세 조회 요청 처리",\n    "status": "완료",\n    "inquiry": "**장기 카드론 두 건의 총 금액과 각 항목별 이자 구분** 확인 요청",\n    "process": [\n        "* 고객 신원 확인 완료 ",\n        * **카드론 건수와 각각의 대출 일자 파악 ("▲▲▲▲ 년도 ▲▲ 월") **,\n          “두 개의 별도 대출 건 존재” 인지함,   \n       *,  대출금 액수 (**▲▲▲▲ 원 / ▲▲▲▲ 원 **) 명확히 함 *,      \n           이자 부과 방식 관련 질문 수렴 *.,     \n             청구 금액 불일치 의심 제기 *,*              \n               현재 상환 계획 기간("12 개월 ") 재확인 *.,                   \n                 세부적인 대출금 납입 이력 미기억 언급 *,                    \n                  각 건 별 청구금액 분리 청구 이유 설명 제공 *" ],,                     "",                        "" ),                         {" result ": "* 고객이 보유 한 장기 카드론 은 과거 에 두 차례 이루어졌으며,\\n각각 다른 시점(\\n「▲▲▲▲ 년 도 ▲▲ 월 」\\ 과 「▲▲▲▲ 년 도 ▲▲ 월 」)\\ n 으로 발급 되어.\\n총 두 건의 대출액 (\\n`▲▲▲▲ 원 ` 와 `\\u304f▲▲▲▲ 원 `) 이며 \\ncurrently 는 분할 상환 중이지만,\\ninitial disbursement amount and interest breakdown details were unclear to the customer.* **\\r\

In [ ]:
metrics_exaone_lg

In [ ]:
process_txt = " ".join(res_exaone_lg['process'])
bert_exaone_lg = calculate_bertscore(summary_ref, process_txt)
bert_exaone_lg

---
clovax 모델은 1차 탈락


In [ ]:
res_clovax, time_clovax = generate_summary_sllm("naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B", system_prompt)

In [46]:
res_clovax

{'error': 'JSON 파싱 실패',
 'raw': '```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제 요청을 하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌번호를 인증할 수 있는지 문의하였고, 불가능하여 다른 인증 방법을 제안받았습니다.",\n    "처리 결과": "고객은 카드 정지 해제를 요청하였고, 상담사는 고객의 요청에 따라 카드를 정지 해제한 후, 즉시 카드 이용이 가능하도록 조치했습니다."\n}\n````````````````````````````````````````````````\nassistant\n```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제를 요청하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌번호를 인증할 수 있는지 문의하였고, 불가능하여 다른 인증 방법을 제안받았습니다.",\n    "처리 결과": "고객은 카드 정지 해제를 요청하였고, 상담사는 고객의 요청에 따라 카드를 정지 해제한 후, 즉시 카드 이용이 가능하도록 조치했습니다."\n}\n``````\nassistant\n```json\n{\n    "상담 유형": "도난 / 분실 신청 / 해제",\n    "문의 사항": "단기 요청한 카드의 정지 해제를 요청하고 있습니다.",\n    "상담 과정": "고객이 단기 요청한 카드 정지 해제를 요청하며, 카드 정보와 본인 확인을 위해 명의자 성함과 카드 번호를 제공했습니다. 이후, 고객은 본인 확인을 위해 핸드폰 번호나 계좌

---

In [2]:
import nltk
from nltk.translate.meteor_score import meteor_score

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Playdata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from konlpy.tag import Okt

okt = Okt()

def calculate_meteor(reference, candidate):
    # ref_tokens = nltk.word_tokenize(reference)
    # cand_tokens = nltk.word_tokenize(candidate)
    
    ref_tokens = okt.morphs(reference)
    cand_tokens = okt.morphs(candidate)
    
    score = meteor_score([ref_tokens], cand_tokens)
    return round(score, 4)

In [ ]:
met_score = calculate_meteor(process_txt, summary_ref)

In [ ]:
met_score

0.3554